In [1]:
import os
import numpy as np
import pandas as pd

from mpire import WorkerPool

RAW_SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/snpDir/"
NEW_SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/offspring/12/snpDir/"
REF_PATH = "/data1/zhoujb/cowpeaSequence/refernce/"

In [2]:
def getNewVarPos(file_name):
    # Load raw vcf file
    vcf_file_path = "/data2/zhoujb/project/cowpea_project/runWES/2_getMut/only_snp.vcf"
    raw_vcf = pd.read_table(vcf_file_path, skiprows=941)

    name_list = []
    with open("/data2/zhoujb/project/cowpea_project/runWES/2_getMut/config_bam") as name_f:
        for line in name_f:
            name_list.append(line.split("\t")[0])

    chr_name_map = {"CM072825.1":"Chr01", "CM072826.1":"Chr02", "CM072827.1":"Chr03", 
                    "CM072828.1":"Chr04", "CM072829.1":"Chr05", "CM072830.1":"Chr06", 
                    "CM072831.1":"Chr07", "CM072832.1":"Chr08", "CM072833.1":"Chr09", 
                    "CM072834.1":"Chr10", "CM072835.1":"Chr11"}
    
    raw_vcf = raw_vcf[raw_vcf["#CHROM"].isin(chr_name_map)].copy()
    raw_vcf["Chr"] = raw_vcf["#CHROM"].map(chr_name_map)

    raw_snp_txt = pd.read_table(os.path.join(RAW_SNP_PATH, file_name))

    vcf_sel = raw_vcf[raw_vcf["Chr"] == file_name.split("_")[0]].copy()

    in_list = list(set(raw_snp_txt["Pos"].to_list()).intersection(vcf_sel["POS"].to_list()))
    out_list = list(set(raw_snp_txt["Pos"].to_list()).difference(vcf_sel["POS"].to_list()))

    vcf_sel = vcf_sel[vcf_sel["POS"].isin(in_list)].copy()

    need_write_list = []
    for index in vcf_sel.index:
        tmp_list = []
        tmp_list.append(vcf_sel.loc[index, "Chr"])
        tmp_list.append(vcf_sel.loc[index, "POS"])

        ref = vcf_sel.loc[index, "REF"]
        alt = vcf_sel.loc[index, "ALT"]
        for item in name_list:
            gt = vcf_sel.loc[index, item].split(":")[0]
            if gt == "0/0":
                out_val = ref
            elif gt == "1/1":
                out_val = alt
            else:
                out_val = "N"

            tmp_list.append(out_val)

        need_write_list.append(tmp_list)
        del tmp_list

    ref_file = pd.read_table(os.path.join(REF_PATH, "cowpeaRef/{}.pos").format(file_name.split(".")[0]), index_col="Pos")
    for item in out_list:
        ref_val = ref_file.loc[item, "Val"]
        tmp_list = [file_name.split("_")[0], item] + [ref_val] * len(name_list)
        need_write_list.append(tmp_list)
        del tmp_list

    out_df = pd.DataFrame(need_write_list, columns=['Chr', 'Pos']+name_list)
    out_df = out_df.sort_values(by=["Pos"], ascending=True)
    out_df.to_csv(os.path.join(NEW_SNP_PATH, file_name), sep="\t", index=0)
    
    return

if __name__ == "__main__":
    file_list = os.listdir(RAW_SNP_PATH)

    with WorkerPool(n_jobs=24) as pool:
        pool.map(getNewVarPos, file_list, progress_bar=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 902/902 [48:07<00:00,  4.03s/it]
